<a href="https://colab.research.google.com/github/immortalrolexx/deepfake_detection/blob/main/deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torch timm

In [2]:
! pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'deepfake-and-real-images' dataset.
Path to dataset files: /kaggle/input/deepfake-and-real-images


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import timm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'deepfake-and-real-images' dataset.
Path to dataset files: /kaggle/input/deepfake-and-real-images


In [6]:
DATA_DIR = '/kaggle/input/deepfake-and-real-images/Dataset'

In [7]:
! ls /kaggle/input/deepfake-and-real-images

Dataset


In [8]:

BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 20
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
IMG_SIZE = 299

In [9]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [10]:
train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'Train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'Validation'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'Test'), transform=transform)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [12]:
print(f"Classes found: {train_dataset.classes}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")

Classes found: ['Fake', 'Real']
Training images: 140002
Validation images: 39428


In [13]:
class DeepfakeEnsemble(nn.Module):
    def __init__(self, num_classes=2):
        super(DeepfakeEnsemble, self).__init__()

        self.effnet = timm.create_model('efficientnet_b4', pretrained=True, num_classes=0)

        for param in self.effnet.parameters():
            param.requires_grad = False

        self.effnet_head = nn.Sequential(
            nn.Linear(1792, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

        self.xception = timm.create_model('xception', pretrained=True, num_classes=0)

        for param in self.xception.parameters():
            param.requires_grad = False

        self.xception_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        eff_logits = self.effnet_head(eff_features)

        xcp_features = self.xception(x)
        xcp_logits = self.xception_head(xcp_features)

        avg_logits = (eff_logits + xcp_logits) / 2.0

        return avg_logits

In [14]:
model = DeepfakeEnsemble(num_classes=2).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


In [15]:
import copy
import numpy

class EarlyStopping:
    def __init__(self, patience=3, min_delta=0, path='best_model.pth'):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)

        elif val_loss > self.best_loss + self.min_delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        self.best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), self.path)
        print(f'Validation loss decreased. Saving model...')

In [16]:
def train_model(model, train_loader, val_loader, epochs):
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}


    early_stopper = EarlyStopping(patience=3, path='best_deepfake_ensemble.pth')

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 10)

        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, leave=True)

        for images, labels in loop:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            loop.set_description(f"Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader)
        history['train_loss'].append(epoch_loss)

        model.eval()
        val_running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_val_loss = val_running_loss / len(val_loader)
        val_acc = 100 * correct / total

        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(val_acc)

        print(f"Train Loss: {epoch_loss:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        early_stopper(epoch_val_loss, model)

        if early_stopper.early_stop:
            print("Early stopping triggered! Restoring best weights...")
            model.load_state_dict(early_stopper.best_model_wts)
            break

    if not early_stopper.early_stop:
        model.load_state_dict(early_stopper.best_model_wts)

    return history

In [17]:
history = train_model(model, train_loader, val_loader, EPOCHS)


Epoch 1/20
----------


  0%|          | 0/4376 [00:00<?, ?it/s]

Train Loss: 0.3212 | Val Loss: 0.3623 | Val Acc: 83.78%
Validation loss decreased. Saving model...

Epoch 2/20
----------


  0%|          | 0/4376 [00:00<?, ?it/s]

Train Loss: 0.2726 | Val Loss: 0.3212 | Val Acc: 86.00%
Validation loss decreased. Saving model...

Epoch 3/20
----------


...

Train Loss: 0.1701 | Val Loss: 0.3124 | Val Acc: 87.54%
EarlyStopping counter: 1 out of 3

Epoch 9/20
----------


  0%|          | 0/4376 [00:00<?, ?it/s]

Train Loss: 0.1598 | Val Loss: 0.3230 | Val Acc: 87.24%
EarlyStopping counter: 2 out of 3

Epoch 10/20
----------


  0%|          | 0/4376 [00:00<?, ?it/s]

Train Loss: 0.1538 | Val Loss: 0.3197 | Val Acc: 87.66%
EarlyStopping counter: 3 out of 3
Early stopping triggered! Restoring best weights...


In [18]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Final Test Accuracy: {100 * correct / total:.2f}%")

  0%|          | 0/341 [00:00<?, ?it/s]

Final Test Accuracy: 74.31%


In [19]:
torch.save(model.state_dict(), 'deepfake_ensemble_model.pth')